# chat UI in fastHTML
> chat UI implemented in fastHTML

You can start to implement fake chat log generator.

In [ ]:
#| default_exp chat_ui

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
from fasthtml.jupyter import *

In [ ]:
#| export
from fasthtml.common import *

In [ ]:
#| export
# Set up the app, including daisyui and tailwind for the chat component
hdrs = (picolink, Script(src="https://cdn.tailwindcss.com"),
        Link(rel="stylesheet", href="https://cdn.jsdelivr.net/npm/daisyui@4.11.1/dist/full.min.css"))
app = FastHTML(hdrs=hdrs)

## Chat components

Basic chat UI components can include Chat Message and a Chat Input. For a Chat Message, the important attributes are the actual message (str) and the role of the message owner (user - boolean value whether the owner is the user, not the AI assistant).

In [ ]:
#| export
# Chat message component (renders a chat bubble)
def ChatMessage(msg, user):
    paragraphs = msg.split("\n")
    # Set class to change displayed style of bubble
    bubble_class = "chat-bubble-primary" if user else 'chat-bubble-secondary'
    chat_class = "chat-end" if user else 'chat-start'
    return  Div(cls=f"chat {chat_class}")(
                Div('User' if user else 'Assistant', cls="chat-header"),
                Div(
                    *[P(p, style=f"color: {'black' if user else 'white'};") for p in paragraphs], 
                    cls=f"chat-bubble {bubble_class}", 
                    style=f"background-color: {'#03fcad' if user else '#025238'};"),
                Hidden(msg, name="messages"),  # Hidden field for submitting past messages to form
                Hidden("user" if user else "assistant", name="roles")  # Hidden field for submitting corresponding owners
            )

For the chat input, set the name for submitting a new message via form.

In [ ]:
#| export
# The input field for the user message. Also used to clear the
# input field after sending a message via an OOB swap
def ChatInput():
    return Input(name='msg', id='msg-input', placeholder="Type a message",
                 cls="input input-bordered w-full rounded-l-2xl bg-stone-800", 
                 hx_swap_oob='true'  # Re-render the element to remove submitted message
                )

### Action Buttons

Simple actions for creating a new message from the user side.

In [ ]:
def ActionButton(content: str):
    return Form(
        hx_post="/",
        hx_target="#chatlist",
        hx_swap="beforeend",  # Location: just before the end of element
    )(
        Hidden(content, name="msg"),
        Button(content, cls="btn btn-secondary")
    )

def ActionPanel():
    return Div(
        ActionButton("Do something"),
        ActionButton("A different action"),
        cls="flex flex-row h-fit px-24 gap-4 pt-4"
    )

## Router

### Home page
The home page should contain our message list and the Chat Input. The main page can be extracted by accessing the index (root) endpoint.

In [ ]:
#| export
@app.get('/')
def index():
    sidebar = Div(
        H1("Conversations"),
        cls="w-[30vw] bg-stone-800"
    )
    page =  Div(cls="w-full flex flex-col p-0")(
        ActionPanel(),
        Form(
            hx_post="/",  # Operation: some POST endpoint with function `send` 
            hx_target="#chatlist",  # Target: element with ID 'chatlist'
            hx_swap="beforeend",  # Location: just before the end of element
            cls="w-full flex flex-col px-24 h-[90vh]"
        )(
            # The chat list
            Div(id="chatlist", cls="chat-box overflow-y-auto flex-1 w-full mt-10")(
                # One initial message from AI assistant
                ChatMessage("Hello! I'm a chatbot. How can I help you today?", False),
            ),
            # Input form
            Div(cls="h-fit mb-5 mt-5 flex space-x-2 mt-2")(
                Group(
                    ChatInput(), 
                    Button("Send", cls="btn btn-primary rounded-r-2xl", style="background-color: #03fcad;"))
            )
        )   
    )
    return Main(
        sidebar,
        page, 
        data_theme="forest", 
        cls="h-[100vh] w-full relative flex flex-row items-stretch overflow-hidden transition-colors z-0 p-0",)

### Form submission

At submission, this function should:

- Extract the new and all previous chat history  
- Prompt & get answers from ChatGPT from all these messages  
- Return a new ChatMessage

In [ ]:
#| export

# Handle the form submission
@app.post('/')
def send(msg: str, messages: list[str] = None, roles: list[str] = None):
    if not messages: messages = []
    if not roles: roles = []

    # Format the prompt for ChatGPT
    prompt = [ {"role": roles[i], "content": messages[i]} for i in range(len(messages)) ]
    
    # Add the user message to the prompt
    prompt.append({"role": "user", "content": msg})
    
    # Get the response from ChatGPT
    response = "Response cannot be generated at the moment, please comeback later."
    
    return (ChatMessage(msg, True),    # The user's message
            ChatMessage(response, False), # The chatbot's response
            ChatInput()) # And clear the input field via an OOB swap

In [ ]:
#| eval: false
#| hide
#| export

serve()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()